In [2]:
import tensorflow as tf 
import os
from random import shuffle
import numpy as np
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

2022-03-20 08:30:21.678966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
os.chdir('/root/fish_class')
working_directory = os.getcwd()
print("working directory:", working_directory)

working directory: /root/fish_class


1. Loading Data and Preprocessing

In [4]:
# 20% Validation Set, 80% Training Set
# Input data is balanced across the number of fish classes
# Following generators and preprocessing used to help generalize the model
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, # min-max Normalization, shifting pixel value to [0,1], max 255 to max of 1 (domain shift)
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    shear_range=0.2, 
    zoom_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(                                                    
    rescale=1./255 # Apply same normalization, not performing other preprocessing steps
)

# train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=tf.keras.applications.vgg16.preprocess_input, # Preprocessing function
#     validation_split=0.2 
# )

# test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=tf.keras.applications.vgg16.preprocess_input # Preprocessing function
# )

In [5]:
# Shuffle = True randomly selects images from a random directory/class to meet the streaming batch size and send to the model for training
# Instead of flow_from_directory, the following article: https://www.kaggle.com/pavfedotov/fish-classifier-efficientnet-acc-100, uses flow_from_dataframe
# which simply contains the list of all image paths in directory and the corresponding class label, we can pivot to this method if it is difficult
# to visualize results, but the method below is actually more efficient...
train_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation' # Will only take 20% of the total data as the validation data
)

test_images = test_generator.flow_from_directory(
    directory= './Data/Test',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False, # We need to be able to generate metrics at the end
    seed=42
)

Found 7215 images belonging to 9 classes.
Found 1802 images belonging to 9 classes.
Found 492 images belonging to 9 classes.


In [6]:
print("Training image shape:", train_images.image_shape)
print("Validation image shape:", val_images.image_shape)
print("Test image shape:", test_images.image_shape)

Training image shape: (300, 300, 3)
Validation image shape: (300, 300, 3)
Test image shape: (300, 300, 3)


In [7]:
train_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [8]:
val_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [9]:
test_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt Head Bream': 1,
 'Horse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [10]:
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2. Defining VGG16 (CNN) Architecture

In [11]:
# Novel model - add descriptive layer names?
input = Input(shape =(300,300,3))
l1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(input)
l2 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(l1)
l3 = MaxPool2D(2,2)(l2)

l4 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(l3)
l5 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(l4)
l6 = MaxPool2D(2,2)(l5)

l7 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(l6)
l8 = MaxPool2D(2,2)(l7)
l9 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(l8)
l10 = MaxPool2D(2,2)(l9)

l11 = Flatten()(l10)
l12 = Dense(128, activation='relu')(l11)
l13 = Dropout(0.4)(l12)
output = Dense(9, activation='softmax')(l13)
model = Model (inputs=input, outputs=output)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 298, 298, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 145, 145, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 72, 64)        0     

2022-03-20 08:30:27.878920: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-20 08:30:27.880537: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-20 08:30:27.910033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 08:30:27.911054: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-03-20 08:30:27.911115: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-03-20 08:30:27.914449: I tensorflow/stream_executor/platform/default/dso_lo

3. Defining Schedulers and Callbacks

In [12]:
# We should be monitoring validation loss calculation not validation accuracy in our callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10) # Fine tune
checkpoint_path = "ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt"
monitor = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                                             verbose=1,save_best_only=True,
                                             save_weights_only=True,
                                             mode='min') # Only saves the best model (so far) in terms of min validation loss
# # Learning rate schedule
# def scheduler(epoch, lr): # Fine tune
#     if epoch%10 == 0: # Occurs on 10, 20, 30, 40, 50
#         lr = lr/2 
#     return lr

# lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=5, min_lr=0.0000001, verbose=1)
callbacks = [early_stop, monitor, lr_schedule]

4. Training Model

In [131]:
history = model.fit(
    train_images, 
    validation_data=val_images, 
    epochs=50, # Fine tune
    callbacks=callbacks
)

226/226 [==============================] - 287s 1s/step - loss: 1.1101 - accuracy: 0.6046 - val_loss: 1.1468 - val_accuracy: 0.5605

Epoch 00001: val_loss improved from 1.81213 to 1.14681, saving model to ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt


In [132]:
np.save('ENEL645_FinalProject_FishClassification/history.npy', history.history)

In [ ]:
model.save('ENEL645_FinalProject_FishClassification/Model_rof')

In [ ]:
print("\n************************ COMPLETED TRAINING ************************")

5. Loading Best Model and Testing

In [19]:
model.load_weights(checkpoint_path)

In [133]:
history=np.load('ENEL645_FinalProject_FishClassification/history.npy', allow_pickle='TRUE').item()
print("Best training results:\n", history)

Best training results:
 {'loss': [1.1101419925689697], 'accuracy': [0.6045737862586975], 'val_loss': [1.1468136310577393], 'val_accuracy': [0.5604883432388306], 'lr': [1e-04]}


In [20]:
results = model.evaluate(test_images, verbose=1)

print("Categorical Cross Entropy: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

10/16 [=================>............] - ETA: 11s - loss: 2.1988 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
print("\n************************ COMPLETED TESTING ************************")